In [1]:
import torch
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

/mnt/disk1/sumdev/CourseWork/a3_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load training dataset, only the first 50,000 examples
train_dataset = load_dataset("wmt16", "de-en", split="train[:100000]")

# Load validation and test datasets
validation_dataset = load_dataset("wmt16", "de-en", split="validation")
test_dataset = load_dataset("wmt16", "de-en", split="test")

# Setup-2A


In [3]:
from typing import Iterable
import torch
import math
import torch.nn as nn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List


# Define function to yield tokens from the dataset
def yield_tokens(data_iter: Iterable, language: str):
    for data_sample in data_iter:
        yield token_transform[language](data_sample['translation'][language])

# Set up tokenizers
token_transform = {}
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')


# Build vocabulary
vocab_transform = {}
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'de'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'en'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)

In [4]:
def sequential_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

# function to add BOS/EOS and create tensor for input sequence indices
def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]),
                      torch.tensor(token_ids),
                      torch.tensor([EOS_IDX])))
    
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequential_transforms(token_transform[ln], #Tokenization
                                               vocab_transform[ln], #Numericalization
                                               tensor_transform) # Add BOS/EOS and create tensor

In [5]:
from torch import Tensor
import torch
import torch.nn as nn
from torch.nn import Transformer
import math

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# helper Module that adds positional encoding to the token embedding to introduce a notion of word order.
class PositionalEncoding(nn.Module):
    def __init__(self,
                 emb_size: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

# helper Module to convert tensor of input indices into corresponding tensor of token embeddings
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, emb_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.emb_size = emb_size

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.emb_size)

# Seq2Seq Network
class Seq2SeqTransformer(nn.Module):
    def __init__(self,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 emb_size: int,
                 nhead: int,
                 src_vocab_size: int,
                 tgt_vocab_size: int,
                 dim_feedforward: int = 512,
                 dropout: float = 0.1):
        super(Seq2SeqTransformer, self).__init__()
        self.transformer = Transformer(d_model=emb_size,
                                       nhead=nhead,
                                       num_encoder_layers=num_encoder_layers,
                                       num_decoder_layers=num_decoder_layers,
                                       dim_feedforward=dim_feedforward,
                                       dropout=dropout)
        self.generator = nn.Linear(emb_size, tgt_vocab_size)
        self.src_tok_emb = TokenEmbedding(src_vocab_size, emb_size)
        self.tgt_tok_emb = TokenEmbedding(tgt_vocab_size, emb_size)
        self.positional_encoding = PositionalEncoding(
            emb_size, dropout=dropout)

    def forward(self,
                src: Tensor,
                trg: Tensor,
                src_mask: Tensor,
                tgt_mask: Tensor,
                src_padding_mask: Tensor,
                tgt_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        src_emb = self.positional_encoding(self.src_tok_emb(src))
        tgt_emb = self.positional_encoding(self.tgt_tok_emb(trg))
        outs = self.transformer(src_emb, tgt_emb, src_mask, tgt_mask, None,
                                src_padding_mask, tgt_padding_mask, memory_key_padding_mask)
        return self.generator(outs)

    def encode(self, src: Tensor, src_mask: Tensor):
        return self.transformer.encoder(self.positional_encoding(
                            self.src_tok_emb(src)), src_mask)

    def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
        return self.transformer.decoder(self.positional_encoding(
                          self.tgt_tok_emb(tgt)), memory,
                          tgt_mask)

In [6]:
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'
vocab_transform = {}
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
vocab_transform[SRC_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'de'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[TGT_LANGUAGE] = build_vocab_from_iterator(yield_tokens(train_dataset, 'en'), specials=["<unk>", "<pad>", "<bos>", "<eos>"])
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_IDX)
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_IDX)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])
EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
BATCH_SIZE = 128
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_loaded = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)
model = model_loaded.to(device)
model_loaded.load_state_dict(torch.load('2A_final_model.pth'))

<All keys matched successfully>

In [7]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

# function to generate output sequence using greedy algorithm
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
    for i in range(max_len-1):
        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0))
                    .type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == EOS_IDX:
            break
    return ys

# actual function to translate input sentence into target language
def translate(model: torch.nn.Module, src_sentence: str):
    model.eval()
    src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
    num_tokens = src.shape[0]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
    tgt_tokens = greedy_decode(
        model,  src, src_mask, max_len=num_tokens + 5, start_symbol=BOS_IDX).flatten()
    return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

In [8]:
sentence = 'Obama empfängt Netanyahu'
actual = 'Obama receives Netanyahu'

translate(model, sentence)

' Is there at present a minimum level of transport'

In [9]:
import pandas as pd

df_path = 'input.csv'
out_path = 'output.csv'
df = pd.read_csv(df_path, sep=',')

translated = []
for i in range(len(df)):
    translated.append(translate(model_loaded, df['de'][i]))

df['en'] = pd.Series(translated)
df.to_csv(out_path, index=False)

In [10]:
print(translate(model_loaded, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A group of people is still a minimum level of transport . 


# Setup-2B

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model = model.to(device)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
prefix = "translate English to German: "
sentence = "The house is wonderful."
input_ids = tokenizer(prefix + sentence, return_tensors="pt").input_ids
outputs = model.generate(input_ids.to(device))
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Das Haus ist wunderbar.


In [13]:
import pandas as pd

df_path = 'input.csv'
out_path = 'output.csv'
df = pd.read_csv(df_path, sep=',')

translated = []
for i in range(len(df)):
    prefix = "translate English to German: "
    sentence = df['de'][i]
    input_ids = tokenizer(prefix + sentence, return_tensors="pt").input_ids
    outputs = model.generate(input_ids.to(device))
    english_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translated.append(english_translation)

df['en'] = pd.Series(translated)
df.to_csv(out_path, index=False)

# Setup-2C

In [24]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, T5Config, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

# Load the T5 tokenizer and model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model_loaded = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small")
model_loaded = model_loaded.to(device)
model_loaded.load_state_dict(torch.load('q2c_t5_model.pth'))

<All keys matched successfully>

In [25]:
actual = 'Obama receives Netanyahu' 
sentence = 'Obama empfängt Netanyahu'

input_ids = tokenizer(sentence, return_tensors="pt").input_ids
input_ids = input_ids.to(device)
outputs = model_loaded.generate(input_ids)
print('=> Translated to English:')
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

=> Translated to English:
Obama senounces Netanyahu


In [11]:
import pandas as pd

df_path = 'input.csv'
out_path = 'output.csv'
df = pd.read_csv(df_path, sep=',')

translated = []
for i in range(len(df)):
    sentence = df['de'][i]
    input_ids = tokenizer(sentence, return_tensors="pt").input_ids
    outputs = model_loaded.generate(input_ids.to(device))
    english_translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    translated.append(english_translation)

df['en'] = pd.Series(translated)
df.to_csv(out_path, index=False)